In [1]:
!pip install geopandas fiona 
!pip install fastkml pykml
from fastkml import kml
from pykml import parser
import re

     |████████████████████████████████| 972kB 5.4MB/s 
     |████████████████████████████████| 14.8MB 321kB/s 
     |████████████████████████████████| 6.5MB 38.9MB/s 
     |████████████████████████████████| 71kB 3.7MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for fastkml: filename=fastkml-0.11-cp36-none-any.whl size=61869 sha256=c31abe5ae93aa9d104e0ef0a08a74ae5dcd69e472a284b208fd915066b835442
  Stored in directory: /root/.cache/pip/wheels/55/01/ea/6191eb73e0894743d02b33a2b1a570e85242844d810804fbf2
  Created wheel for pygeoif: filename=pygeoif-0.7-cp36-none-any.whl size=19249 sha256=5aaba90023d3f9ead1a0aa1ac51b3028a66ce805bc5e40284a7ac57c84f4a58e
  Stored in directory: /root/.cache/pip/wheels/60/6e/a7/3d3eef59ac84a86663d0f5c5a92091f5056e9aeb6588c4de34
Successfully built fastkml pygeoif


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Goal here is to read in & access this ~1gb file
#File is in this area (of my drive, at least): /content/drive/My Drive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_grip4
# Trying: https://gis.stackexchange.com/questions/32762/how-to-access-feature-classes-in-file-geodatabases-with-python-and-gdal (look far down)

#Data taken from: https://www.globio.info/download-grip-dataset
#Some similar data here: https://enterprisecontentnew-usfs.hub.arcgis.com/datasets/motor-vehicle-use-map-roads-feature-layer?page=17611


import fiona 
import geopandas as gpd

# Get all the layers from the .gdb file 
#%%%%%%%%%%%%%%%%% We May Have Different Drive Paths To This Folder %%%%%%%%%%%%%%%%%%%%
gdb_file_path =  '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/GRIP4_GlobalRoads.gdb'
# Vlado's Path: gdb_file_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/GRIP4_region1.gdb'


water_bodies_path =  '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/hydropolys.gdb'
linear_water_path =  '/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/hydrolines.gdb'
#water_bodies_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/hydropolys.gdb'
#linear_water_path = '/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/hydrolines.gdb'


#Get our datasets ready for reading:

c = fiona.open(gdb_file_path, 'r')

print(len(c))

linear_water = fiona.open(linear_water_path, 'r')
poly_water = fiona.open(water_bodies_path, 'r')

#gdf = gpd.read_file(gdb_file_path)
#gdf.head(2)

25758453


In [4]:
c[6]['geometry']['coordinates']

[[(-123.15683999999999, 49.15667000000008),
  (-123.15675999999996, 49.14998000000003),
  (-123.15675999999996, 49.14206000000007),
  (-123.15674999999999, 49.134930000000054)]]

In [5]:
import datetime
now = datetime.datetime.now()
print(now)

'''
Finds the indeces of c that are within our desired range
'''
def linear_grid_finder(clist,lat_min,lat_max,long_min,long_max, kind):
  idcs = []
  for i in range(1,len(clist)-1):
      if kind!=1:
        #print("latitude:", lat_min, clist[i]['geometry']['coordinates'][0][0][1], lat_max)
        #print("longitude:", long_min, clist[i]['geometry']['coordinates'][0][0][0], long_max)
        if lat_min < clist[i]['geometry']['coordinates'][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0] < long_max:
            idcs.append(i)
      else:
         if lat_min < clist[i]['geometry']['coordinates'][0][0][0][1] < lat_max:
          if long_min < clist[i]['geometry']['coordinates'][0][0][0][0] < long_max:
            idcs.append(i)
  return idcs

now = datetime.datetime.now()
print(now)

2020-12-02 16:29:08.052726
2020-12-02 16:29:08.053081


In [7]:
'''
Splits a rectangular surface into smaller rectangles and returns them all
'''
def split_grid(num_rows,num_cols,x_min,x_max,y_min,y_max):
    quad_list = []
    increment_rows = (y_max - y_min)/num_rows
    increment_cols = (x_max - x_min)/num_cols

    for i in range(1,num_rows):
      for j in range(1,num_cols):
          quad_list.append((x_min+(i-1)*increment_cols, x_min+(i)*increment_cols, y_min+(j-1)*increment_rows, y_min+(j)*increment_rows))
    return quad_list

In [ ]:
print(split_grid(10,10,0,100,0,100))

[(0.0, 10.0, 0.0, 10.0), (0.0, 10.0, 10.0, 20.0), (0.0, 10.0, 20.0, 30.0), (0.0, 10.0, 30.0, 40.0), (0.0, 10.0, 40.0, 50.0), (0.0, 10.0, 50.0, 60.0), (0.0, 10.0, 60.0, 70.0), (0.0, 10.0, 70.0, 80.0), (0.0, 10.0, 80.0, 90.0), (10.0, 20.0, 0.0, 10.0), (10.0, 20.0, 10.0, 20.0), (10.0, 20.0, 20.0, 30.0), (10.0, 20.0, 30.0, 40.0), (10.0, 20.0, 40.0, 50.0), (10.0, 20.0, 50.0, 60.0), (10.0, 20.0, 60.0, 70.0), (10.0, 20.0, 70.0, 80.0), (10.0, 20.0, 80.0, 90.0), (20.0, 30.0, 0.0, 10.0), (20.0, 30.0, 10.0, 20.0), (20.0, 30.0, 20.0, 30.0), (20.0, 30.0, 30.0, 40.0), (20.0, 30.0, 40.0, 50.0), (20.0, 30.0, 50.0, 60.0), (20.0, 30.0, 60.0, 70.0), (20.0, 30.0, 70.0, 80.0), (20.0, 30.0, 80.0, 90.0), (30.0, 40.0, 0.0, 10.0), (30.0, 40.0, 10.0, 20.0), (30.0, 40.0, 20.0, 30.0), (30.0, 40.0, 30.0, 40.0), (30.0, 40.0, 40.0, 50.0), (30.0, 40.0, 50.0, 60.0), (30.0, 40.0, 60.0, 70.0), (30.0, 40.0, 70.0, 80.0), (30.0, 40.0, 80.0, 90.0), (40.0, 50.0, 0.0, 10.0), (40.0, 50.0, 10.0, 20.0), (40.0, 50.0, 20.0, 30.0),

In [8]:
#idcs = linear_grid_finder(c,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max,0)

def return_coords(clist, idx, kind):
  if kind == 1:
     return clist[idx]['geometry']['coordinates'][0][0]
  else:
     return clist[idx]['geometry']['coordinates'][0]

def return_all_coords(clist, idcs, kind=0):
  all_coords=[]
  for idx in idcs:
      if clist[idx]:
        if len(clist[idx]['geometry']['coordinates'])==1:
          all_coords.append(return_coords(clist,idx, kind))
        else:
          for i in range(len(clist[idx]['geometry']['coordinates'])):
              all_coords.append(clist[idx]['geometry']['coordinates'][i])
  return all_coords

In [9]:
import matplotlib.pyplot as plt

def prepare_plot(roads_data):
  prepared_dataX = []
  prepared_dataY = []
  for feature in roads_data:
    these_xs = []
    these_ys = []
    try:
      for x, y in feature:#apparently this is faster than *zip(): https://stackoverflow.com/questions/8081545/how-to-convert-list-of-tuples-to-multiple-lists
        these_xs.append(x)
        these_ys.append(y)
    except:
      for x, y in feature[0]:
        these_xs.append(x)
        these_ys.append(y)

    #print("prepared_dataX receives", these_xs)
    prepared_dataX.append(these_xs)

    #print("prepared_dataY receives", these_ys)
    prepared_dataY.append(these_ys)
  return prepared_dataX, prepared_dataY

#All done. Now we can graph this stuff

In [10]:
from matplotlib.pyplot import figure
from matplotlib import pyplot as plt
plt.ioff()
import numpy as np
def create_image(roadX, roadY, waterX_poly, waterY_poly, waterX_lin, waterY_lin, n):
  fig = figure(num=None, figsize=(10, 10), dpi=100, facecolor='w', edgecolor='k') #Resize to 1000x1000 images
  if roadX:
    for i in range(0, len(roadX)):
      tuple_setX = roadX[i]
      tuple_setY = roadY[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 2, color = 'grey')
  if waterX_lin:
    for i in range(0, len(waterX_lin)):
      tuple_setX = waterX_lin[i]
      tuple_setY = waterY_lin[i]
      for j in range(0, len(tuple_setX)-1, 1):
        plt.plot(tuple_setX[j:j+2], tuple_setY[j:j+2], '-', linewidth = 8, color = 'blue')
  if waterX_poly:
      for i in range(len(waterX_poly)):
          plt.fill(waterX_poly[i], waterY_poly[i], edgecolor='blue', facecolor='blue')
  plt.axis('off')
  #plt.savefig('/content/drive/My Drive/ENGS108_Final_Project/Data_Sets/our_images/roads_' +  str(n) + '.png', bbox_inches = 'tight')
  plt.savefig('/content/drive/MyDrive/College Year 3/20F/ENGS108/ENGS108_Final_Project/Data_Sets/roads_and_water_dataset/' + 'roads_and_water' + str(n) + '.png', bbox_inches = 'tight')

In [11]:
##Need a roads filter to delete some of the extra roads to reduce linear_grid_finder search time:

def filter_roads(goal_lat_min, goal_lat_max, goal_long_min, goal_long_max, dataset):
  filtered_dataset = []
  for i in range(1,len(dataset),1):
      if goal_lat_min < dataset[i]['geometry']['coordinates'][0][0][1] < goal_lat_max:
        if goal_long_min < dataset[i]['geometry']['coordinates'][0][0][0] < goal_long_max:
          filtered_dataset.append(dataset[i])
  return filtered_dataset



In [ ]:
goal_lat_min = 44 #bounds for germany/france/netherlands/belgium
goal_lat_max = 52 # next up: https://keras.io/examples/generative/pixelcnn/
x_cuts = int(abs(goal_lat_max-goal_lat_min)*50)
goal_long_min = 3
goal_long_max = 10
y_cuts = int(abs(goal_long_max-goal_long_min)*50)
list_of_coords = split_grid(x_cuts,y_cuts,goal_lat_min,goal_lat_max, goal_long_min, goal_long_max)
print("# of cubes we are splitting up:", len(list_of_coords))
filtered_roads = filter_roads(goal_lat_min, goal_lat_max, goal_long_min, goal_long_max, c)
print("finally finished filtering roads. length was reduced dramatically:", len(c), len(filtered_roads))
for k in range(2400,len(list_of_coords)):

    min_lat, max_lat, min_long, max_long = list_of_coords[k]

    idcs = linear_grid_finder(filtered_roads, min_lat, max_lat, min_long, max_long, 0)
    if len(idcs) > 0:

      print(k)
      all_coords = return_all_coords(filtered_roads,idcs)
      idcs1 = linear_grid_finder(poly_water, min_lat, max_lat, min_long, max_long, 1)
      lake_coords = return_all_coords(poly_water, idcs1,1)
      idcs2 = linear_grid_finder(linear_water,  min_lat, max_lat, min_long, max_long, 2)
      river_coords = return_all_coords(linear_water, idcs2,2)

      roadX,roadY = prepare_plot(all_coords)
      wpolyX, wpolyY = prepare_plot(lake_coords)
      wlinX, wlinY = prepare_plot(river_coords)


      create_image(roadX,roadY, wpolyX, wpolyY, wlinX, wlinY, k)
    else:
      print("no roads detected on this interval:", min_lat, max_lat, min_long, max_long)

# of cubes we are splitting up: 139251
finally finished filtering roads. length was reduced dramatically: 25758453 1627935
no roads detected on this interval: 44.137142857142855 44.16 8.355 8.3725
no roads detected on this interval: 44.137142857142855 44.16 8.3725 8.39
no roads detected on this interval: 44.137142857142855 44.16 8.39 8.4075
no roads detected on this interval: 44.137142857142855 44.16 8.4075 8.425
no roads detected on this interval: 44.137142857142855 44.16 8.425 8.4425
no roads detected on this interval: 44.137142857142855 44.16 8.4425 8.46
no roads detected on this interval: 44.137142857142855 44.16 8.46 8.477500000000001
no roads detected on this interval: 44.137142857142855 44.16 8.477500000000001 8.495000000000001
no roads detected on this interval: 44.137142857142855 44.16 8.495000000000001 8.5125
no roads detected on this interval: 44.137142857142855 44.16 8.5125 8.530000000000001
no roads detected on this interval: 44.137142857142855 44.16 8.530000000000001 8.54

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


no roads detected on this interval: 44.16 44.182857142857145 4.365 4.3825
no roads detected on this interval: 44.16 44.182857142857145 4.3825 4.4
no roads detected on this interval: 44.16 44.182857142857145 4.4 4.4175
no roads detected on this interval: 44.16 44.182857142857145 4.4175 4.4350000000000005
2525
2526
2527
no roads detected on this interval: 44.16 44.182857142857145 4.4875 4.505
no roads detected on this interval: 44.16 44.182857142857145 4.505 4.5225
no roads detected on this interval: 44.16 44.182857142857145 4.5225 4.54
2531
2532
2533
2534
2535
2536
no roads detected on this interval: 44.16 44.182857142857145 4.6450000000000005 4.6625
no roads detected on this interval: 44.16 44.182857142857145 4.6625 4.68
no roads detected on this interval: 44.16 44.182857142857145 4.68 4.6975
no roads detected on this interval: 44.16 44.182857142857145 4.6975 4.715
no roads detected on this interval: 44.16 44.182857142857145 4.715 4.7325
no roads detected on this interval: 44.16 44.182